In [1]:
import gudhi
import numpy as np
import torch

In [2]:
X = np.random.rand(30, 4)

In [3]:
landmarks = gudhi.pick_n_random_points(points=X, nb_points=10)

In [5]:
witness_complex = gudhi.EuclideanWitnessComplex(witnesses=X, landmarks=landmarks)
simplex_tree = witness_complex.create_simplex_tree(3)

In [8]:
simplex_tree.compute_persistence()

In [9]:
simplex_tree.betti_numbers()

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [10]:
simplex_tree.persistence_pairs()

[([8, 4], [8, 4, 2]),
 ([7, 5], [8, 7, 1]),
 ([7, 6], [8, 7, 2]),
 ([7, 6, 5], [8, 7, 5, 2]),
 ([9, 7, 6], [9, 7, 6, 1]),
 ([3], [])]

In [12]:
simplex_tree.persistent_betti_numbers(0, np.inf)

[1, 0, 0, 0, 0, 0, 0, 0, 0]

In [15]:
simplex_tree.persistence()

[(2, (0.09901224753495519, 0.1185795162562594)),
 (2, (0.2384204237953934, 0.2570642980317541)),
 (1, (0.0, 0.02578041406333309)),
 (1, (0.0, 0.021656426719765642)),
 (1, (0.004660972440652278, 0.016511804229204785)),
 (0, (0.0, inf))]

In [17]:
from scipy.spatial import distance_matrix
np.sum(distance_matrix(X, X) == 0)

30

In [19]:
from gudhi.wasserstein.barycenter import lagrangian_barycenter
from functional.homology import drop_inf
X, Y, Z = np.random.rand(20, 3), np.random.rand(20, 3), np.random.rand(20, 3)
from gph import ripser_parallel
dX, dY, dZ = drop_inf(ripser_parallel(X)['dgms']), drop_inf(ripser_parallel(Y)['dgms']), drop_inf(ripser_parallel(Z)['dgms'])

In [22]:
ret = lagrangian_barycenter([dX[1], dY[1], dZ[1]])
ret

array([[0.57148115, 0.6056848 ],
       [0.48694547, 0.52946866],
       [0.52784425, 0.65355211],
       [0.34950409, 0.36539868],
       [0.43965062, 0.46321133],
       [0.43726616, 0.46354111],
       [0.62383872, 0.63134642],
       [0.59096758, 0.59415181]])

In [27]:
ret = ripser_parallel(X, return_generators=True, maxdim=4)
dX, gensX = ret['dgms'], ret['gens']

In [36]:
np.repeat(gensX[0][:, :-2], 2, axis=-1)

array([[ 9,  9],
       [10, 10],
       [16, 16],
       [18, 18],
       [ 7,  7],
       [11, 11],
       [13, 13],
       [17, 17],
       [15, 15],
       [12, 12],
       [ 8,  8],
       [ 0,  0],
       [14, 14],
       [ 4,  4],
       [ 6,  6],
       [19, 19],
       [ 3,  3],
       [ 1,  1],
       [ 5,  5]])

In [41]:
from torch.nn.functional import normalize
import torch

normalize(torch.tensor(X)).norm(dim=-1)

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000, 1.0000], dtype=torch.float64)

In [42]:
torch.tensor(X).norm(dim=-1)

tensor([0.8645, 1.0396, 1.1524, 0.3843, 0.8170, 1.3034, 0.6862, 0.8164, 1.0135,
        1.2617, 0.8001, 0.9600, 1.4056, 1.0706, 0.5348, 0.4182, 0.7274, 0.7393,
        0.8543, 1.1643], dtype=torch.float64)

In [43]:
from functional.homology import diagrams

dgms, gens = diagrams(X, gens=True)
dgms, gens

([array([[0.        , 0.11235537],
         [0.        , 0.13567343],
         [0.        , 0.16028726],
         [0.        , 0.16489358],
         [0.        , 0.17940228],
         [0.        , 0.2145982 ],
         [0.        , 0.25935212],
         [0.        , 0.26767468],
         [0.        , 0.2840579 ],
         [0.        , 0.2978334 ],
         [0.        , 0.30338418],
         [0.        , 0.30443987],
         [0.        , 0.33023775],
         [0.        , 0.35590068],
         [0.        , 0.37971938],
         [0.        , 0.39152566],
         [0.        , 0.39631605],
         [0.        , 0.41448182],
         [0.        , 0.49552026],
         [0.        ,        inf]], dtype=float32),
  array([[0.47915885, 0.5152166 ],
         [0.41608992, 0.48677203],
         [0.41099122, 0.48981607]], dtype=float32)],
 (array([[ 9,  9,  5],
         [10, 10,  6],
         [16, 16,  6],
         [18, 18,  7],
         [ 7, 18,  0],
         [11, 11,  1],
         [13, 13,  7],

In [51]:
def diagrams_to_tensor(dgms: [list[np.ndarray], list[list[np.ndarray]]], fill_value=np.nan, requires_grad: bool = False) -> torch.Tensor:
    if isinstance(dgms[0], list):
        m_dgm = max((d for b in dgms for d in b), key=lambda x: x.shape[0]).shape[0]
        return torch.tensor(np.stack(
            [
                np.stack(
                    [
                        np.pad(dgms[b][dim], ((0, m_dgm - dgms[b][dim].shape[0]), (0, 0)), constant_values=fill_value) for dim in range(len(dgms[b]))
                    ]
                ) for b in range(len(dgms))
            ]
        ), requires_grad=requires_grad)

    m_dgm = max(dgms, key=lambda x: x.shape[0]).shape[0]
    return torch.tensor(
        np.stack(
            [
                np.pad(dgms[dim], ((0, m_dgm - dgms[dim].shape[0]), (0, 0)), constant_values=fill_value) for dim in range(len(dgms))
            ]
        ), requires_grad=requires_grad
    )


def gens_to_tensor(gens: [list[np.ndarray], list[list[np.ndarray]]], fill_value: int = -1):
    if isinstance(gens[0], list):
        for b in range(len(gens)):
            gens[b][0] = np.repeat(gens[b][0], [2, 1, 1], axis=-1)
        return diagrams_to_tensor(gens, fill_value=fill_value)
    gens[0] = np.repeat(gens[0], [2, 1, 1], axis=-1)
    return diagrams_to_tensor(gens, fill_value=fill_value)

In [52]:
dgms_tensor, gens_tensor = diagrams_to_tensor(dgms), gens_to_tensor([gens[0], *gens[1]], 0)
dgms_tensor, gens_tensor

(tensor([[[0.0000, 0.1124],
          [0.0000, 0.1357],
          [0.0000, 0.1603],
          [0.0000, 0.1649],
          [0.0000, 0.1794],
          [0.0000, 0.2146],
          [0.0000, 0.2594],
          [0.0000, 0.2677],
          [0.0000, 0.2841],
          [0.0000, 0.2978],
          [0.0000, 0.3034],
          [0.0000, 0.3044],
          [0.0000, 0.3302],
          [0.0000, 0.3559],
          [0.0000, 0.3797],
          [0.0000, 0.3915],
          [0.0000, 0.3963],
          [0.0000, 0.4145],
          [0.0000, 0.4955],
          [0.0000,    inf]],
 
         [[0.4792, 0.5152],
          [0.4161, 0.4868],
          [0.4110, 0.4898],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan,    nan],
          [   nan

In [54]:
gens_birth, gens_death = gens_tensor[:, :, :-2], gens_tensor[:, :, -2:]

In [58]:
X_tensor = torch.tensor(X)
X_tensor[gens_birth[:, :, 1]]

tensor([[[0.7026, 0.0507, 0.3794],
         [0.7026, 0.0507, 0.3794]],

        [[0.5945, 0.4304, 0.0894],
         [0.1970, 0.3294, 0.0188]]], dtype=torch.float64)

In [65]:
normalize(X_tensor[gens_birth[:, :, 1]] - X_tensor[gens_birth[:, :, 0]]).shape

torch.Size([2, 19, 3])

In [99]:
def boundary_matrix(gens: np.ndarray, x: np.ndarray):
    return np.logical_or.reduce(gens.reshape(*gens.shape, 1) == x.reshape(1, -1), axis=-2).astype(int)

In [77]:
(gens_birth.numpy().reshape(*gens_birth.shape, 1) == x.reshape(1, -1)).shape

(2, 19, 2, 20)

In [80]:
x = np.arange(X.shape[0])
Jb = boundary_matrix(gens_birth.numpy(), x)
gb = normalize(X_tensor[gens_birth[:, :, 1]] - X_tensor[gens_birth[:, :, 0]])

Jb.shape, gb.shape

((2, 19, 20), torch.Size([2, 19, 3]))

In [85]:
(gb.unsqueeze(-2) * torch.tensor(Jb).unsqueeze(-1)).sum(dim=-3).sum(dim=-3)

tensor([[ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.8890, -0.2856, -0.1421],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.4410, -0.8024, -0.4482],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.1229, -0.5240,  0.8826],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000],
        [-0.8890, -0.2856, -0.1421],
        [ 0.4410, -0.8024, -0.4482],
        [ 0.1229, -0.5240,  0.8826]], dtype=torch.float64)

In [86]:
def batch_select(X: torch.Tensor, gens: torch.Tensor) -> torch.Tensor:
    ix = torch.tensor(gens)
    dummy = ix.unsqueeze(-1).expand(*ix.shape, X.shape[-1]).squeeze(-2)
    return torch.gather(X.detach().repeat(1, 2, 1, 1), 2, dummy)


def grad_subroutine(X: torch.Tensor, gens: torch.Tensor) -> torch.Tensor:
    boundary1 = boundary_matrix(gens[:, :, :, :1].numpy(force=True), np.arange(X.shape[1]))
    boundary0 = boundary_matrix(gens[:, :, :, 1:].numpy(force=True), np.arange(X.shape[1]))
    J0 = -normalize(batch_select(X, gens[:, :, :, 1]) - batch_select(X, gens[:, :, :, 0])).unsqueeze(-2) * boundary0.unsqueeze(-1)
    J1 = normalize(batch_select(X, gens[:, :, :, 1]) - batch_select(X, gens[:, :, :, 0])).unsqueeze(-2) * boundary1.unsqueeze(-1)
    return J0 + J1

In [90]:
X = torch.rand((1, 20, 4))
gens = [diagrams(X[0].numpy(), gens=True)[1]]
gens_tensor = gens_to_tensor([[gens[0][0], *gens[0][1]]], fill_value=0)
gens_birth, gens_death = gens_tensor[:, :, :, :-2], gens_tensor[:, :, :, -2:]

In [91]:
gens_birth

tensor([[[[18, 18],
          [13, 13],
          [14, 14],
          [15, 15],
          [ 6,  6],
          [ 2,  2],
          [16, 16],
          [ 1,  1],
          [ 5,  5],
          [ 8,  8],
          [ 7,  7],
          [ 3,  3],
          [11, 11],
          [ 9,  9],
          [12, 12],
          [ 0,  0],
          [10, 10],
          [17, 17],
          [19, 19]],

         [[10,  3],
          [10,  6],
          [ 9,  1],
          [19,  8],
          [ 8,  4],
          [16, 11],
          [14,  8],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0]]]])

In [102]:
boundary1 = boundary_matrix(gens_birth[:, :, :, :1].numpy(force=True), np.arange(X.shape[1]))
boundary0 = boundary_matrix(gens_birth[:, :, :, 1:].numpy(force=True), np.arange(X.shape[1]))

In [105]:
boundary0.shape

(1, 2, 19, 20)

In [129]:
norm = normalize(batch_select(X, gens_birth[:, :, :, 1]) - batch_select(X, gens_birth[:, :, :, 0]), dim=-1).unsqueeze(-2)
J0 = -norm * torch.tensor(boundary0).unsqueeze(-1)
J1 = norm * torch.tensor(boundary1).unsqueeze(-1)

/var/folders/gj/6k0s5jts7dv8ygp7t_n5nch80000gn/T/ipykernel_35019/3734665028.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ix = torch.tensor(gens)


In [130]:
J0.shape

torch.Size([1, 2, 19, 20, 4])

In [115]:
gens_birth[:, :, :, 1]

tensor([[[18, 13, 14, 15,  6,  2, 16,  1,  5,  8,  7,  3, 11,  9, 12,  0, 10,
          17, 19],
         [ 3,  6,  1,  8,  4, 11,  8,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
           0,  0]]])

In [137]:
J = J0 + J1
J[0, 1, 0]

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.9141, -0.0181, -0.4040, -0.0284],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [-0.9141,  0.0181,  0.4040,  0.0284],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  0.0000]])

In [138]:
gens_birth[0, 1, 0]

tensor([10,  3])

In [139]:
Jb = J0 + J1

boundary1 = boundary_matrix(gens_death[:, :, :, :1].numpy(force=True), np.arange(X.shape[1]))
boundary0 = boundary_matrix(gens_death[:, :, :, 1:].numpy(force=True), np.arange(X.shape[1]))
norm = normalize(batch_select(X, gens_death[:, :, :, 1]) - batch_select(X, gens_death[:, :, :, 0]), dim=-1).unsqueeze(-2)
J0 = -norm * torch.tensor(boundary0).unsqueeze(-1)
J1 = norm * torch.tensor(boundary1).unsqueeze(-1)
Jd = J0 + J1

/var/folders/gj/6k0s5jts7dv8ygp7t_n5nch80000gn/T/ipykernel_35019/3734665028.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ix = torch.tensor(gens)


In [144]:
(Jd + Jb).sum(dim=1).sum(dim=1)

tensor([[[-0.5928,  0.2714, -0.2291,  0.5668],
         [ 0.4789, -0.3300, -1.3250, -0.8911],
         [ 0.1905,  0.5321, -1.0395,  0.0204],
         [ 1.0429, -0.9780, -0.1956,  0.1075],
         [ 0.4544,  2.0590,  1.4427,  0.1114],
         [-0.7757,  0.9341, -2.2087, -1.3429],
         [-0.4709, -1.2034,  0.8175,  0.4446],
         [-0.1849,  0.0341, -0.7461,  0.6387],
         [-2.2058,  1.3499,  0.6283,  0.8157],
         [ 1.1856,  0.0553,  0.1972,  1.5439],
         [-2.0584, -2.4611, -0.4054,  1.4035],
         [-0.4308,  0.0433, -0.8031, -0.5025],
         [ 1.9227, -1.6619,  0.4221, -0.7756],
         [-1.3274,  0.3054,  0.4503,  0.6140],
         [ 2.0767,  0.3900,  0.0884, -0.5182],
         [ 0.4866, -0.2445,  0.4898, -0.1932],
         [ 0.1782, -0.5841,  1.2706, -0.8757],
         [ 0.2364,  0.8226, -0.1044,  0.5064],
         [-0.4245, -0.3247,  0.7378, -0.4122],
         [ 0.2187,  0.9904,  0.5122, -1.2614]]])

In [145]:
import numpy as np
import torch
from functional.homology import diagrams

X, Y = torch.rand((2, 10, 3)), torch.rand((2, 10, 3))
XX, YY = torch.cdist(X, X), torch.cdist(Y, Y)
inf_block = torch.triu(torch.full_like(XX, torch.inf), 1) + XX
M = torch.cat([
    torch.cat([XX, inf_block.transpose(1, 2), torch.zeros((XX.shape[0], XX.shape[1], 1))], dim=-1),
    torch.cat([inf_block, torch.minimum(XX, YY), torch.full((XX.shape[0], XX.shape[1], 1), torch.inf)], dim=-1),
    torch.cat([torch.zeros((XX.shape[0], 1, XX.shape[1])), torch.full((XX.shape[0], 1, XX.shape[1]), torch.inf), torch.zeros((XX.shape[0], 1, 1))], dim=-1)
], dim=1)
ret = [diagrams(M[b].numpy(force=True), distances=True, gens=True) for b in range(X.shape[0])]

In [146]:
M

tensor([[[0.0000, 0.8475, 0.8059, 1.2511, 0.4667, 0.5052, 0.6020, 1.0950,
          0.6059, 0.5152, 0.0000, 0.8475, 0.8059, 1.2511, 0.4667, 0.5052,
          0.6020, 1.0950, 0.6059, 0.5152, 0.0000],
         [0.8475, 0.0000, 0.7298, 0.4177, 0.4651, 0.3820, 0.4628, 0.2770,
          0.6245, 0.7773,    inf, 0.0000, 0.7298, 0.4177, 0.4651, 0.3820,
          0.4628, 0.2770, 0.6245, 0.7773, 0.0000],
         [0.8059, 0.7298, 0.0000, 0.9066, 0.4571, 0.6158, 0.6094, 0.8212,
          0.2420, 0.6923,    inf,    inf, 0.0000, 0.9066, 0.4571, 0.6158,
          0.6094, 0.8212, 0.2420, 0.6923, 0.0000],
         [1.2511, 0.4177, 0.9066, 0.0000, 0.8233, 0.7708, 0.7954, 0.2072,
          0.8943, 1.1122,    inf,    inf,    inf, 0.0000, 0.8233, 0.7708,
          0.7954, 0.2072, 0.8943, 1.1122, 0.0000],
         [0.4667, 0.4651, 0.4571, 0.8233, 0.0000, 0.2382, 0.2798, 0.6732,
          0.2527, 0.4360,    inf,    inf,    inf,    inf, 0.0000, 0.2382,
          0.2798, 0.6732, 0.2527, 0.4360, 0.0000],
     

In [147]:
dgms, gens = [drop_inf(r[0]) for r in ret], [[r[1][0], *r[1][1]] for r in ret]
gens_tensor = gens_to_tensor(gens, fill_value=0)

gens_birth, gens_death = gens_tensor[:, :, :, :-2], gens_tensor[:, :, :, -2:]
gens

[[array([], shape=(0, 4), dtype=int64),
  array([[17, 12, 19, 17],
         [17, 10, 19, 17],
         [16, 11, 19, 11],
         [15, 11,  5,  1],
         [15, 13, 13, 11],
         [19, 15, 18, 15],
         [19, 18, 18, 16]])],
 [array([], shape=(0, 4), dtype=int64),
  array([[19, 17, 17, 11],
         [12, 11,  4,  2],
         [16, 13, 17, 13],
         [18, 10, 17, 10],
         [19, 11,  7,  3]])]]

In [151]:
gens_tensor.shape

torch.Size([2, 2, 7, 4])

In [150]:
gens = gens_birth
J = torch.zeros((X.shape[0], gens.shape[1], X.shape[1] + Y.shape[1] + 1, gens.shape[2]))
J.shape

torch.Size([2, 2, 21, 7])

In [152]:
def grad_subroutine(X: torch.Tensor, gens: torch.Tensor) -> torch.Tensor:
    boundary1 = boundary_matrix(gens[:, :, :, :1].numpy(force=True), np.arange(X.shape[1]))
    boundary0 = boundary_matrix(gens[:, :, :, 1:].numpy(force=True), np.arange(X.shape[1]))
    norm = normalize(batch_select(X, gens[:, :, :, 1]) - batch_select(X, gens[:, :, :, 0]), dim=-1).unsqueeze(-2)
    J0 = -norm * torch.tensor(boundary0).unsqueeze(-1)
    J1 = norm * torch.tensor(boundary1).unsqueeze(-1)
    return J0 + J1

In [177]:
Z = torch.cat((X, Y), dim=-2)

In [179]:
Z.shape

torch.Size([2, 20, 3])

In [190]:
def batch_select(X: torch.Tensor, ix: torch.Tensor) -> torch.Tensor:
    dummy = ix.unsqueeze(-1).expand(*ix.shape, X.shape[-1]).squeeze(-2)
    return torch.gather(X.detach().unsqueeze(1).repeat(1, ix.shape[1], 1, 1), 2, dummy)

In [189]:
Z.detach().unsqueeze(-3).repeat(1, gens.shape[1], 1, 1).shape

torch.Size([2, 2, 20, 3])

In [196]:
XX[:, :, :, 1]

tensor([[[0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 1, 1, 1, 0]],

        [[0, 0, 0, 0, 0, 0, 0],
         [1, 1, 1, 0, 1, 0, 0]]])

In [203]:
XX = gens % X.shape[0]
# torch.all(gens < X.shape[0], dim=-1).unsqueeze(-1).repeat(1, 1, 1, 2)
X_normed = normalize(batch_select(Z, XX[:, :, :, 1]) - batch_select(Z, XX[:, :, :, 0]), dim=-1)
X_normed.shape

torch.Size([2, 2, 7, 3])

In [204]:
YY = gens % X.shape[0] + X.shape[0]
Y_normed = normalize(batch_select(Z, YY[:, :, :, 1]) - batch_select(Z, YY[:, :, :, 0]), dim=-1)
Y_normed.shape

torch.Size([2, 2, 7, 3])

In [205]:
# gensXY = gens[(gens[:, :, :, 0] < X.shape[0]) & (gens[:, :, :, 1] < 2 * X.shape[0])]
# JXY = grad_subroutine(X, torch.cat((gensXY[:, :, :, 0], gensXY[:, :, :, 1] - X.shape[0]), dim=-1))
# J += torch.scatter(J, 0, gensXY, JXY)
Xnorm = torch.norm(batch_select(Z, XX[:, :, :, 1]) - batch_select(Z, XX[:, :, :, 0]))
Ynorm = torch.norm(batch_select(Z, YY[:, :, :, 1]) - batch_select(Z, YY[:, :, :, 0]))
# gensYYX, gensYYY = gensYY[Xnorm <= Ynorm], gensYY[Xnorm > Ynorm]

In [212]:
torch.all((X.shape[1] <= gens) & (gens < 2 * X.shape[1]), dim=-1)

tensor([[[False, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True,  True,  True]],

        [[False, False, False, False, False, False, False],
         [ True,  True,  True,  True,  True, False, False]]])

In [216]:
((Xnorm <= Ynorm) * X_normed + (X_normed > Y_normed) * Y_normed) * (torch.all((X.shape[1] <= gens) & (gens < 2 * X.shape[1]), dim=-1).unsqueeze(-1))

tensor([[[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[-0.5500, -1.4965, -0.5871],
          [-0.5500, -1.4965, -0.5871],
          [ 0.6307,  0.5940,  1.0102],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.5500, -1.4965, -0.5871]]],


        [[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0776, -0.8682, -0.5811],
          [ 0.0776, -0.8682, -0.5811],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.000

In [222]:
torch.all((X.shape[1] > gens[:, :, :, :1]) & (X.shape[1] <= gens[:, :, :, 1:]) & (gens[:, :, :, 1:] < 2 * X.shape[1]), dim=-1).unsqueeze(-1)

tensor([[[[False],
          [False],
          [False],
          [False],
          [False],
          [False],
          [False]],

         [[False],
          [False],
          [False],
          [False],
          [False],
          [False],
          [False]]],


        [[[False],
          [False],
          [False],
          [False],
          [False],
          [False],
          [False]],

         [[False],
          [False],
          [False],
          [False],
          [False],
          [False],
          [False]]]])

In [224]:
X_normed * (torch.all((X.shape[1] > gens[:, :, :, :1]) & (gens[:, :, :, 1:] < 2 * X.shape[1]), dim=-1).unsqueeze(-1))

tensor([[[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[-0., -0., -0.],
          [-0., -0., -0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [-0., -0., -0.]]],


        [[[0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]],

         [[0., 0., 0.],
          [0., -0., 0.],
          [0., -0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.],
          [0., 0., 0.]]]])

In [175]:
torch.nn.functional.normalize(torch.full((1, 3), torch.inf))

tensor([[nan, nan, nan]])

In [153]:
gens[torch.all(gens < X.shape[0], dim=-1)]

tensor([[0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0],
        [0, 0]])

In [155]:
gensXX = gens[torch.all(gens < X.shape[0], dim=-1)]
JXX = grad_subroutine(X, gensXX)
JXX

IndexError: too many indices for tensor of dimension 2

In [ ]:
J += torch.scatter(J, 0, gensXX, JXX)

In [236]:
def cross_grad_subroutine(X: torch.Tensor, Y: torch.Tensor, gens: torch.Tensor) -> torch.Tensor:
    Z = torch.cat((X, Y), dim=-2)
    boundary1 = boundary_matrix(gens[:, :, :, :1].numpy(force=True), np.arange(Z.shape[1] + 1))
    boundary0 = boundary_matrix(gens[:, :, :, 1:].numpy(force=True), np.arange(Z.shape[1] + 1))

    XX = gens % X.shape[0]
    X_normed = normalize(batch_select(Z, XX[:, :, :, 1]) - batch_select(Z, XX[:, :, :, 0]), dim=-1)
    YY = gens % X.shape[0] + X.shape[0]
    Y_normed = normalize(batch_select(Z, YY[:, :, :, 1]) - batch_select(Z, YY[:, :, :, 0]), dim=-1)
    Xnorm = torch.norm(batch_select(Z, XX[:, :, :, 1]) - batch_select(Z, XX[:, :, :, 0]))
    Ynorm = torch.norm(batch_select(Z, YY[:, :, :, 1]) - batch_select(Z, YY[:, :, :, 0]))
    JYY = ((Xnorm <= Ynorm) * X_normed + (X_normed > Y_normed) * Y_normed) * torch.all((X.shape[1] <= gens) & (gens < 2 * X.shape[1]), dim=-1).unsqueeze(-1)
    JXXY = X_normed * torch.all((X.shape[1] > gens[:, :, :, :1]) & (gens[:, :, :, 1:] < 2 * X.shape[1]), dim=-1).unsqueeze(-1)

    J0 = -(JYY + JXXY).unsqueeze(-2) * torch.tensor(boundary0).unsqueeze(-1)
    J1 = (JYY + JXXY).unsqueeze(-2) * torch.tensor(boundary1).unsqueeze(-1)
    return J0 + J1

In [226]:
X

tensor([[[2.6840e-02, 2.5122e-01, 1.7465e-01],
         [4.9297e-01, 7.5462e-01, 6.7228e-01],
         [7.3086e-01, 8.0014e-02, 5.2756e-01],
         [8.0408e-01, 8.9828e-01, 9.1109e-01],
         [3.6583e-01, 3.5090e-01, 4.7961e-01],
         [3.7979e-01, 5.5890e-01, 3.6436e-01],
         [2.3528e-01, 3.7398e-01, 7.2595e-01],
         [6.2327e-01, 7.9709e-01, 9.1305e-01],
         [4.9785e-01, 1.4424e-01, 5.4041e-01],
         [5.4199e-02, 1.1296e-01, 6.7022e-01]],

        [[2.4599e-01, 9.9228e-01, 2.0872e-01],
         [7.6543e-01, 7.9373e-02, 2.5754e-01],
         [6.4467e-01, 3.5094e-02, 9.7140e-01],
         [1.6117e-01, 7.9888e-01, 2.4277e-01],
         [8.2537e-01, 4.6742e-01, 5.1712e-01],
         [8.3253e-01, 5.8205e-01, 5.4622e-01],
         [8.4203e-01, 4.0801e-01, 3.4573e-01],
         [6.4442e-01, 6.6971e-01, 6.0695e-01],
         [6.3397e-01, 1.6715e-01, 1.8709e-01],
         [8.1015e-04, 4.1153e-01, 1.1509e-02]]])

In [227]:
XX, YY = torch.cdist(X, X), torch.cdist(Y, Y)
inf_block = torch.triu(torch.full_like(XX, torch.inf), 1) + XX
M = torch.cat([
    torch.cat([XX, inf_block.transpose(1, 2), torch.zeros((XX.shape[0], XX.shape[1], 1))], dim=-1),
    torch.cat([inf_block, torch.minimum(XX, YY), torch.full((XX.shape[0], XX.shape[1], 1), torch.inf)], dim=-1),
    torch.cat([torch.zeros((XX.shape[0], 1, XX.shape[1])), torch.full((XX.shape[0], 1, XX.shape[1]), torch.inf), torch.zeros((XX.shape[0], 1, 1))], dim=-1)
], dim=1)
ret = [diagrams(M[b].numpy(force=True), distances=True, gens=True) for b in range(X.shape[0])]

In [237]:
dgms, gens = [drop_inf(r[0]) for r in ret], [[r[1][0], *r[1][1]] for r in ret]
dgms_tensor, gens_tensor = diagrams_to_tensor(dgms, requires_grad=False), gens_to_tensor(gens, fill_value=0)

gens_birth, gens_death = gens_tensor[:, :, :, :-2], gens_tensor[:, :, :, -2:]

In [232]:
gens_birth.sort()

torch.return_types.sort(
values=tensor([[[[ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0]],

         [[12, 17],
          [10, 17],
          [11, 16],
          [11, 15],
          [13, 15],
          [15, 19],
          [18, 19]]],


        [[[ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0],
          [ 0,  0]],

         [[17, 19],
          [11, 12],
          [13, 16],
          [10, 18],
          [11, 19],
          [ 0,  0],
          [ 0,  0]]]]),
indices=tensor([[[[0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1]],

         [[1, 0],
          [1, 0],
          [1, 0],
          [1, 0],
          [1, 0],
          [1, 0],
          [1, 0]]],


        [[[0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1],
          [0, 1]],

  

In [238]:
gens_birth.sort()
gens_death.sort()
Jb = cross_grad_subroutine(X, Y, gens_birth)
Jd = cross_grad_subroutine(X, Y, gens_death)

torch.Size([2, 2, 7, 3]) torch.Size([2, 2, 7, 3]) (2, 2, 7, 21) (2, 2, 7, 21)
torch.Size([2, 2, 7, 3]) torch.Size([2, 2, 7, 3]) (2, 2, 7, 21) (2, 2, 7, 21)


In [243]:
J = (Jd + Jb).sum(dim=1).sum(dim=1)
J[:, :X.shape[0]], J[:, X.shape[0]:-1]

(tensor([[[0., 0., 0.],
          [0., 0., 0.]],
 
         [[0., 0., 0.],
          [0., 0., 0.]]]),
 tensor([[[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.5500,  1.4965,  0.5871],
          [-0.6307, -0.5940, -1.0102],
          [ 0.5500,  1.4965,  0.5871],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [-0.6307, -0.5940, -1.0102],
          [ 0.6307,  0.5940,  1.0102],
          [-1.1000, -2.9929, -1.1743],
          [ 1.1807,  2.0904,  1.5973],
          [-0.5500, -1.4965, -0.5871]],
 
         [[ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]

In [1]:
import numpy as np
import gudhi as gd

image = np.random.randn(3, 4, 5)

In [2]:
image

array([[[-1.09339157, -0.05570947, -1.09945945, -0.13949959,
          0.76180344],
        [-0.71850734,  1.41897674, -0.04368329,  0.04951572,
         -0.23245784],
        [ 0.52294593, -1.66958524,  0.21220995,  0.63077788,
         -1.20341585],
        [-0.36083429,  1.12797683, -0.75196499, -0.67847315,
         -0.46731045]],

       [[ 1.24797946, -0.10136017, -0.68942729, -0.4709564 ,
          0.43289292],
        [ 0.1641202 ,  0.17778304,  0.2825157 , -0.1605319 ,
          0.77389963],
        [-0.2211865 , -1.05962684, -0.15067392,  0.53626729,
          0.62235003],
        [-1.49650804,  0.74228548, -0.42473942,  0.11401305,
         -1.04786836]],

       [[ 1.37585486,  0.04120063, -1.3921501 ,  0.40755423,
          0.80592876],
        [-0.19594017,  0.20166782, -0.97877864, -1.99817797,
         -0.51531589],
        [-0.78746784,  0.43480311,  0.59210004, -1.8561321 ,
         -0.32893349],
        [-0.23711509,  0.86996393, -0.90367346, -1.49067626,
          0

In [3]:
c = gd.CubicalComplex(vertices=image)

In [4]:
ret = c.persistence()
ret

[(1, (0.1641201970555368, 1.247979464467818)),
 (1, (0.049515722352663875, 0.6307778805565699)),
 (1, (-0.16053190382509688, 0.2825157028778877)),
 (1, (0.1777830392065117, 0.5362672890498609)),
 (1, (0.016023161562854434, 0.114013054474488)),
 (1, (0.2122099494344993, 0.2825157028778877)),
 (0, (-1.9981779667589703, inf)),
 (0, (-1.6695852359560355, -0.15067392113281913)),
 (0, (-1.496508038843792, -0.22118649918292627)),
 (0, (-1.0933915735773376, -0.055709469338955035)),
 (0, (-1.203415848379495, -0.4247394235440138)),
 (0, (-1.0478683607102353, -0.46731044713307884)),
 (0, (-0.7874678448783485, -0.23711508863981806)),
 (0, (-1.3921500957668813, -0.9787786357441106)),
 (0, (-1.099459454899001, -0.6894272859580431)),
 (0, (-0.7519649902038066, -0.46731044713307884))]

In [5]:
c.vertices()

array([[[-1.09339157, -0.05570947, -1.09945945, -0.13949959,
          0.76180344],
        [-0.71850734,  1.41897674, -0.04368329,  0.04951572,
         -0.23245784],
        [ 0.52294593, -1.66958524,  0.21220995,  0.63077788,
         -1.20341585],
        [-0.36083429,  1.12797683, -0.75196499, -0.67847315,
         -0.46731045]],

       [[ 1.24797946, -0.10136017, -0.68942729, -0.4709564 ,
          0.43289292],
        [ 0.1641202 ,  0.17778304,  0.2825157 , -0.1605319 ,
          0.77389963],
        [-0.2211865 , -1.05962684, -0.15067392,  0.53626729,
          0.62235003],
        [-1.49650804,  0.74228548, -0.42473942,  0.11401305,
         -1.04786836]],

       [[ 1.37585486,  0.04120063, -1.3921501 ,  0.40755423,
          0.80592876],
        [-0.19594017,  0.20166782, -0.97877864, -1.99817797,
         -0.51531589],
        [-0.78746784,  0.43480311,  0.59210004, -1.8561321 ,
         -0.32893349],
        [-0.23711509,  0.86996393, -0.90367346, -1.49067626,
          0

In [6]:
c.betti_numbers()

[1, 0, 0, 0]

In [1]:
# Example from README
from nn.loss.ph import PQLoss
import torch, numpy as np, matplotlib.pyplot as plt

# random pointcloud
np.random.seed(0)
data = np.random.rand(1, 100, 2)

# optimization to increase size of holes
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = PQLoss(2, 0)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(x)
    loss.backward()
    optimizer.step()

In [3]:
# save figure
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)

y = x[0].detach().numpy()
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].scatter(data[0, :,0], data[0, :,1])
ax[0].set_title("Before")
ax[1].scatter(y[:,0], y[:,1])
ax[1].set_title("After")
for i in range(2):
    ax[i].set_yticklabels([])
    ax[i].set_xticklabels([])
    ax[i].tick_params(bottom=False, left=False)
plt.savefig('holes.png')

In [4]:
# Example from README
from nn.loss.ph import PQLoss
import torch, numpy as np, matplotlib.pyplot as plt

# random pointcloud
np.random.seed(0)
data = np.random.rand(1, 100, 2)

# optimization to increase size of holes
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = PQLoss(2, 1)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(x)
    loss.backward()
    optimizer.step()
    
y = x[0].detach().numpy()
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].scatter(data[0, :,0], data[0, :,1])
ax[0].set_title("Before")
ax[1].scatter(y[:,0], y[:,1])
ax[1].set_title("After")
for i in range(2):
    ax[i].set_yticklabels([])
    ax[i].set_xticklabels([])
    ax[i].tick_params(bottom=False, left=False)
plt.savefig('holes1.png')

In [5]:
# Example from README
from nn.loss.ph import PQLoss
import torch, numpy as np, matplotlib.pyplot as plt

# random pointcloud
np.random.seed(0)
data = np.random.rand(1, 100, 2)

# optimization to increase size of holes
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = PQLoss(4, 2)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(x)
    loss.backward()
    optimizer.step()
    
y = x[0].detach().numpy()
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].scatter(data[0, :,0], data[0, :,1])
ax[0].set_title("Before")
ax[1].scatter(y[:,0], y[:,1])
ax[1].set_title("After")
for i in range(2):
    ax[i].set_yticklabels([])
    ax[i].set_xticklabels([])
    ax[i].tick_params(bottom=False, left=False)
plt.savefig('holes2.png')

In [6]:
# Example from README
from nn.loss.ph import PQLoss
import torch, numpy as np, matplotlib.pyplot as plt

# random pointcloud
np.random.seed(0)
data = np.random.rand(1, 100, 2)

# optimization to increase size of holes
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = PQLoss(0, 2)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(x)
    loss.backward()
    optimizer.step()
    
y = x[0].detach().numpy()
fig, ax = plt.subplots(ncols=2, figsize=(10,5))
ax[0].scatter(data[0, :,0], data[0, :,1])
ax[0].set_title("Before")
ax[1].scatter(y[:,0], y[:,1])
ax[1].set_title("After")
for i in range(2):
    ax[i].set_yticklabels([])
    ax[i].set_xticklabels([])
    ax[i].tick_params(bottom=False, left=False)
plt.savefig('holes3.png')

In [10]:
ax = fig.add_subplot(projection='3d')

# Example from README
from nn.loss.ph import PQLoss
import torch, numpy as np, matplotlib.pyplot as plt

# random pointcloud
np.random.seed(0)
data = np.random.randn(1, 100, 3)

# optimization to increase size of holes
x = torch.autograd.Variable(torch.tensor(data).type(torch.float), requires_grad=True)
f1 = PQLoss(2, 0)
optimizer = torch.optim.Adam([x], lr=1e-2)
for i in range(100):
    optimizer.zero_grad()
    loss = -f1(x)
    loss.backward()
    optimizer.step()

In [11]:
y = x[0].detach().numpy()
fig = plt.figure(figsize=(10,5))

ax = fig.add_subplot(1, 2, 1, projection='3d')
ax.scatter(data[0, :,0], data[0, :,1])
ax.set_title("Before")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.tick_params(bottom=False, left=False)

ax = fig.add_subplot(1, 2, 2, projection='3d')
ax.scatter(y[:,0], y[:,1])
ax.set_title("After")
ax.set_yticklabels([])
ax.set_xticklabels([])
ax.tick_params(bottom=False, left=False)

plt.savefig('holes5.png')

In [13]:
f1.right

0